# АНАЛИЗ БАЗЫ ДАННЫХ СЕРВИСА ДЛЯ ЧТЕНИЯ КНИГ

Компания купила крупный сервис для чтения книг по подписке. Задача — проанализировать базу данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задачи анализа:

- посчитать, сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- определить автора с самой высокой средней оценкой кни, учитывая только книги с 50 и более оценками;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Прочитаем таблицы из базы.

In [2]:
query = '''
SELECT *
FROM books
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [4]:
query = '''
SELECT *
FROM authors
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [5]:
query = '''
SELECT *
FROM ratings
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
query = '''
SELECT *
FROM reviews
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

## ЗАДАЧА 1
Рассмотрим, сколько книг вышло после 1 января 2000 года.

In [7]:
query = '''
SELECT COUNT(book_id) AS books_cnt
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,books_cnt
0,819


После 1 января 2000 года вышло 819 книг.

## ЗАДАЧА 2
Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [8]:
query = '''
SELECT books.title, 
    COUNT(distinct reviews.review_id) AS reviews_cnt, 
    AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.book_id
ORDER BY COUNT(reviews.review_id) DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

,title,reviews_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
5,Angels & Demons (Robert Langdon #1),5,3.678571
6,Harry Potter and the Order of the Phoenix (Har...,5,4.186667
7,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
8,Animal Farm,5,3.729730
9,The Fellowship of the Ring (The Lord of the Ri...,5,4.391892


Больше всего оборов у книги "Twilight (Twilight #1)" со средним рейтингом 3.66.

## ЗАДАЧА 3
Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, тем самым исключим из анализа брошюры:

In [9]:
query = '''
SELECT publisher,
    COUNT(book_id) AS books_cnt
FROM publishers
LEFT JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE num_pages > 50
GROUP BY publisher
ORDER BY books_cnt DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,books_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Наибольшее число книг толще 50 страниц выпустило издательство *Penguin Books* (42).

## ЗАДАЧА 4
Определим автора с самой высокой средней оценкой книг(будем учитывать только книги с 50 и более оценками):

In [10]:
query = '''
SELECT author,
    AVG(sub.avg_rating) AS high_avg_rating
FROM
    (SELECT author,
        books.book_id,
        AVG(rating) AS avg_rating
    FROM authors
    LEFT JOIN books ON books.author_id = authors.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY author,
        books.book_id
    HAVING COUNT(rating) > 50) AS sub
GROUP BY author
ORDER BY high_avg_rating DESC
'''

pd.io.sql.read_sql(query, con = engine)

,author,high_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самую высокую среднюю оценку книг (4.28) среди книг с 50 и более оценками получила Джоан Роулинг.

## ЗАДАЧА 5
Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [11]:
query = '''
SELECT AVG(sub_review_cnt.review_cnt) AS avg_review_cnt
FROM
    (SELECT COUNT(text) AS review_cnt
    FROM reviews
    WHERE username IN
        (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(ratings) > 50)
    GROUP BY username
    ) AS sub_review_cnt  
'''

pd.io.sql.read_sql(query, con = engine)

,avg_review_cnt
0,24.333333


Пользователи, поставившие более 50 оценок, в среднем написали по 24 обзора.

# ВЫВОД

Мы провели анализ базы данных по заданным параметрам.

После 1 января 2000 года вышло **819 книг**.

Больше всего оборов у книги **"Twilight (Twilight #1)"** со средним рейтингом 3.66.

Наибольшее число книг толще 50 страниц выпустило издательство ***Penguin Books*** (42).

Самую высокую среднюю оценку книг (4.28) среди книг с 50 и более оценками получила **Джоан Роулинг**.

Пользователи, поставившие более 50 оценок, в среднем написали **по 24 обзора**.